In [ ]:
!pip install torch==2.7.0 --index-url https://download.pytorch.org/whl/cu118

In [ ]:
import torch
from transformers import pipeline, set_seed
from googletrans import Translator


set_seed(42)

In [8]:
device = torch.device("cpu")

In [9]:
generator = pipeline("text-generation", model="kurakurai/Luth-LFM2-700M", trust_remote_code=True, device=device)

Device set to use cpu


In [10]:
translator = Translator()


async def translate_text_from_english_to_russian(text: str):
    coroutine = await translator.translate(text, dest="ru")
    return coroutine.text


async def translate_text_from_russian_to_english(text: str):
    coroutine = await translator.translate(text, dest="en")
    return coroutine.text

In [ ]:
async def _generate(prompt: str, question: str):

    prompt += question

    coroutine = translate_text_from_russian_to_english(prompt)
    prompt = await coroutine

    message = [
        {"role": "user", "content": prompt},
    ]

    generated_answer = generator(message)
    generated_answer = generated_answer[0]['generated_text']

    answer_assistant = ""
    for item in generated_answer:
        if item["role"] == "assistant":
            answer_assistant = item["content"]
            break

    coroutine = translate_text_from_english_to_russian(answer_assistant)
    answer_assistant = await coroutine

    return answer_assistant

In [ ]:
from prompts.science import question, query


async def add_prompts(prompt: str):
    science = await _generate(prompt, question)
    print(science)
    return query

In [ ]:
async def generate(prompt: str):

    prompt += add_prompts()

    coroutine = translate_text_from_russian_to_english(prompt)
    prompt = await coroutine

    message = [
        {"role": "user", "content": prompt},
    ]

    generated_answer = generator(message)
    generated_answer = generated_answer[0]['generated_text']

    answer_assistant = ""
    for item in generated_answer:
        if item["role"] == "assistant":
            answer_assistant = item["content"]
            break

    coroutine = translate_text_from_english_to_russian(answer_assistant)
    answer_assistant = await coroutine

    return answer_assistant

In [ ]:
prompt = "Как у тебя дела?"


answer = generate(prompt)
print(answer)

У меня все хорошо, спасибо! А ты?
